In [ ]:
#참고

#https://github.com/jaara/AI-blog/blob/master/CartPole-A3C.py 의 코드를
#보면서 첨삭하는 식으로 짰습니다.

#공부한 사이트
'''
https://jaromiru.com/2017/03/26/lets-make-an-a3c-implementation/
https://www.youtube.com/watch?v=gINks-YCTBs&t=3531s
https://github.com/XenderLiu/Policy-Gradient-and-Actor-Critic-Keras/blob/master/agent_dir/agent_actorcritic.py
https://github.com/rlcode/reinforcement-learning-kr/blob/master/2-cartpole/2-actor-critic/cartpole_a2c.py#L79
http://www.modulabs.co.kr/RL_library/3305
'''

#gradient관련

'''
https://wikidocs.net/8281
http://blog.naver.com/PostView.nhn?blogId=atelierjpro&logNo=220978848453&parentCategoryNo=&categoryNo=&viewDate=&isShowPopularPosts=false&from=postView
'''


In [38]:
import time
import numpy as np
import random
from keras.layers import Dense,merge
from keras.models import Sequential ,Input, Model
from keras.optimizers import RMSprop
from keras import backend as K
import tensorflow as tf
import gym
import threading

In [39]:
GAMMA = 0.99
N_STEP_RETURN = 8
GAMMA_N = GAMMA ** N_STEP_RETURN
state_size = 4
action_size = 2
reward_size = 1

LOSS_V = .5 # v loss coefficient
LOSS_ENTROPY = .01 # entropy coefficient
MIN_BATCH = 32
NONE_STATE = np.zeros(state_size)
name = 'CartPole-v1'

class Agent:
    #environment와 상호작용으로 얻은 data를 memory에 담고
    #처리해 brain으로 push
    def __init__(self):
        self.memory = []
        
        self.epsilon = 1
        self.decaying_per = 0.999
        
        self.R = 0 #이후에 받을수 있는 모든 Reward 는 R, 현상황에 받는 reward는 reward로 표기하였다.
        
    def getEpsilon(self):
        self.epsilon *= self.decaying_per #expliot and exploration
        return self.epsilon
        
    def act(self,state,actor):
        #decaying e를 통해 expliot and exploration는 하지만
        #목적함수에 cross entropy로 exploration이 일어나도록 했는데, 또 해야하는지는 의문
        #random.choice로 주는 것도 확률을 두번일으키게 하는 것이라, 수정할까 고민함.
        if(random.random() > self.getEpsilon()):
            return random.randint(0,action_size-1)
        else :
            state = np.array([state])
            
            percent = actor.predict_action(state)
            action = np.random.choice(action_size,p=percent[0])
            return action
    
    def train(self, state, action, reward, next_state):
        #한번 행동후 sars 모두 memory에 저장,
        #Reward 계산.
        #game이 done됐을시 get_sample 함수를 통해
        #next_state를 수정하며,Reward 계산 후 brain으로 현재 쌓인학습데이터를 모두 보냄
        
        # N_STEP_RETURN보다 memory의 길이가 길어졌을 시,
        # next_state 수정하며,Reward 계산 후, memory 비움
        def get_sample(memory,n):
            state, action, _, _ =memory[0]
            _, _, _, next_state = memory[n-1]
            return state, action, self.R, next_state
        
        action_onehot = np.zeros(action_size)
        action_onehot[action] = 1
        
        self.memory.append((state,action_onehot,reward,next_state))
        
        self.R = (self.R + reward * GAMMA_N) / GAMMA
        
        if next_state is None :
            
            while len(self.memory) > 0 :
                n = len(self.memory)
                state, action, reward, next_state = get_sample(self.memory,n)
                brain.train_push(state, action, reward, next_state)
                
                self.R = (self.R - self.memory[0][2]) / GAMMA
                self.memory.pop(0)
                
            self.R = 0
            
        if len(self.memory) >= N_STEP_RETURN : 
            state, action, reward, next_state = get_sample(self.memory, N_STEP_RETURN)
            brain.train_push(state, action, reward, next_state)
            
            self.R = self.R - self.memory[0][2]
            self.memory.pop(0)
            

In [40]:

class Brain:
    '''
    각 Environment의 agents 로부터 정보를 받아 한꺼번에 처리함으로서 학습데이터간의 종속성 해결
    tensorflow session은 오류검색해서 session등록만 해주고 다 backend K로 통일해 함수사용했음.
    
    '''
    #train_queue는 각 Environment의 agent의 state, action, reward, next_state, next_state의 상태를 담음
    train_queue = [ [], [], [], [], [] ]
    lock_queue = threading.Lock()
    def __init__(self):
        #먼저 기존 tensorflow 에서 세션을 만든다. 
        #그리고 그것을 keras에 등록시켜 keras가 해당 세션을 사용할 수 있도록 한다.
        self.session = tf.Session()
        K.set_session(self.session)
        
        #직접 만든 tensor를 쓰기위해선 initialization을 해야함
        K.manual_variable_initialization(True)
        self.actor_lr = 0.003
        
        #policy 를 actor_model을 통해 근사시킴
        self.model = self.actor_model()
        _,_,_,self.object_function = self.build_object_function(self.model)
        
        #변수를 만들었기 때문에 initializer를 run해줘야함
        self.session.run(tf.global_variables_initializer())
        self.default_graph = tf.get_default_graph()
        
    def load_model(self,dir):
        self.model.load_weights(dir)
        
    def save_model(self,dir):
        self.model.save_weights(dir)
        
    def actor_model(self):
        #actor의 policy를 근사하는 output action과
        #critic역할을 하는 action_value를 return
        input_layer = Input(batch_shape = (None,state_size))
        h1 = Dense(32, activation = 'relu')(input_layer)
        action = Dense(action_size,activation = 'softmax')(h1)
        action_value = Dense(1,activation='linear')(h1)
        model = Model(inputs = input_layer , outputs=[action,action_value])
        return model
    
    def build_object_function(self,model):
        #placeholder 생성
        state_tensor = K.placeholder(dtype = 'float32', shape = (None,state_size))
        action_tensor = K.placeholder(dtype = 'float32',shape = (None,action_size))
        reward_tensor = K.placeholder(dtype = 'float32',shape=(None,reward_size))
        #model로부터 action확률과, value를 tensor로 받아옴.
        probability, value = model(state_tensor)
        #probability가 0이 될까봐 1e-10을 더함
        log_probability = K.log(K.sum(probability * action_tensor\
                                             ,keepdims=True)+1e-10)
        #reward - base value로 분산을 낮춰줌
        advantage = reward_tensor - value
        #policy 를 근사할땐 advantage gradient의 update는 멈춰줘야함
        loss_policy = - log_probability * K.stop_gradient(advantage)
        #value는 advantage값의 ^2에 계수를 곱
        loss_value = K.constant(LOSS_V) * K.square(advantage)
        #exploration 을위해 entropy
        entropy = K.constant(LOSS_ENTROPY) * K.sum(probability * K.log(probability+1e-10),\
                                             axis = 1, keepdims = True)
        #최종 loss function
        loss_total = K.mean(loss_policy + loss_value + entropy)
        
        optimizer = RMSprop(lr=self.actor_lr, decay=.99)
        updates = optimizer.get_updates(model.trainable_weights,[],loss_total)
        train = K.function([state_tensor,action_tensor,reward_tensor],[],updates=updates)
        return state_tensor, action_tensor, reward_tensor, train
    
    def optimize(self):
        #MIN_BATCH 보다 train_queue가 작으면 return
        #MIN_BATCH* 5 보다 처리해야할 양이 많으면 들어오는 batch를 줄이던지,
        #optimizer갯수를늘림
        #object_function 으로 보냄
        if len(self.train_queue[0])< MIN_BATCH:
            return
        with self.lock_queue:
            if len(self.train_queue[0]) < MIN_BATCH:# more thread could have passed without lock
                return 
        
            state, action, reward, next_state, state_mask = self.train_queue
            self.train_queue = [[],[],[],[],[]]
        #학습데이터를 np.vstack으로 쌓음
        state = np.vstack(state)
        action = np.vstack(action)
        reward = np.vstack(reward)
        next_state = np.vstack(next_state)
        state_mask = np.vstack(state_mask)
        
        if len(state) > 5 * MIN_BATCH:
            print("minimizing batch")
        # train data로부터는 N_STEP_RETURN 이후의 value를 못받으므로 더해줘야함
        value = self.predict_value(next_state)
        
        reward = reward + GAMMA_N * value * state_mask
        self.object_function([state,action,reward])
    
    
    def train_push(self,state, action, reward, next_state):
        #train_queue로 학습데이터를 보냄
        with self.lock_queue:
            self.train_queue[0].append(state)
            self.train_queue[1].append(action)
            self.train_queue[2].append(reward)

            if next_state is None:
                self.train_queue[3].append(NONE_STATE)
                self.train_queue[4].append(0.)

            else :
                self.train_queue[3].append(next_state)
                self.train_queue[4].append(1.)

    def predict(self, state): 
        with self.default_graph.as_default():
            action, value = self.model.predict(state)
            return action,value

    def predict_action(self, state): 
        with self.default_graph.as_default():
            action, value = self.model.predict(state)
            return action

    def predict_value(self, state): 
        with self.default_graph.as_default():
            action, value = self.model.predict(state)
            return value

In [41]:

class Environment(threading.Thread):
    #환경을 thread로 여러개돌리기위해 class로만듬
    stop_signal = False
    
    def __init__(self,game_name,render = False):
        threading.Thread.__init__(self)
        
        self.render = render
        self.env = gym.make(game_name)
        self.agent = Agent()
        
    def run_episode(self):
        state = self.env.reset()
        R = 0

        while True:
            if self.render :
                self.env.render()
            action = self.agent.act(state,brain)

            next_state, reward, done, info = self.env.step(action)
            reward = reward if not done or R == 499 else -100

            if done :
                next_state = None

            self.agent.train(state,action,reward,next_state)

            state = next_state
            R += reward
            if done :
                break
        
    def run(self):
        while not self.stop_signal:
            self.run_episode()
    
    def stop(self):
        self.stop_signal = True
                
class Optimizer(threading.Thread):
    #optimizer를 thread로 여러개돌리기위해 class화
    stop_signal = False
    def __init__(self):
        threading.Thread.__init__(self)
    def run(self):
        while not self.stop_signal:
            brain.optimize()
                    
    def stop(self):
        self.stop_signal = True

In [42]:
env_test = Environment(render=True,game_name = name)

In [51]:
agent = Agent()
brain = Brain()

In [52]:
#brain.load_model('10minute_model.h5')

In [47]:
THREADS = 7
OPTIMIZERS = 2

In [53]:
envs = [Environment(game_name = name) for i in range(THREADS)]
opts = [Optimizer() for i in range(OPTIMIZERS)]

In [54]:
for e in envs:
    e.start()
for o in opts:
    o.start()

In [56]:
for e in envs:
    e.stop()
for e in envs:
    e.join()
for o in opts:
    o.stop()
for o in opts:
    o.join()

In [58]:
#brain.save_model("10minute_model.h5")

In [25]:
env_test.run()

KeyboardInterrupt: 